# CAVEAT

As of now, 26.01.2025, the scraper is not working anymore. The reason being that Autovit.ro changed their class names and structure of the website. Since we parsed the website using class names, the scraper is not able to extract the information anymore. The code is still here for future reference and for the sake of completeness.

You can run it but a dataset.csv empty file will be generated.

# Imports

In [1]:
from time import sleep
import pandas as pd
import requests
from bs4 import BeautifulSoup
from loguru import logger

# Scraper class

In [2]:
class AutoVitScraper:
    def __init__(self):
        self.df = None
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
        }

    def get_urls(self):
        urls = set()
        for i in range(2):
            url = f"https://www.autovit.ro/autoturisme?search%5Bfilter_enum_damaged%5D=0&page={i}"
            response = requests.get(url, headers=self.headers)
            if response.status_code != 200:
                logger.error(f"Failed page {i}: {response.status_code}")
                continue

            soup = BeautifulSoup(response.text, "lxml")
            sections = soup.find_all("section", class_="ooa-qat6iw ecru18x1")

            for section in sections:
                urls.add(section.find("p", class_="e2z61p70 ooa-1ed90th er34gjf0").find("a")["href"])

        self.df = pd.DataFrame(urls, columns=["url"])

    @staticmethod
    def path(url):
        return url[url.find("anunt/") + len("anunt/") : url.find(".html")]

    @staticmethod
    def id(url):
        return AutoVitScraper.path(url).split("-")[-1]

    def prepare_dataframe(self):
        self.df["id"] = self.df["url"].apply(self.id)
        self.df["path"] = self.df["url"].apply(self.path)

        columns_to_add = [
            "audio si tehnologie",
            "confort si echipamente optionale",
            "electronice si sisteme de asistenta",
            "performanta",
            "siguranta",
            "vehicule electrice",
        ]
        for col in columns_to_add:
            self.df[col] = None

    def fetch_all_details(self):
        for index, row in self.df.iterrows():
            self.fetch_details(index, row)

    def fetch_details(self, index, row):
        url = row["url"]
        response = None

        for _ in range(3):
            try:
                sleep(1)
                response = requests.get(url, headers=self.headers)
                if response.status_code == 200:
                    break
            except requests.exceptions.ConnectionError:
                logger.error(f"DNS issues {url}")

        if not response or response.status_code != 200:
            return

        soup = BeautifulSoup(response.text, "lxml")

        # Process general information
        info = self.general_info(soup)
        if info:
            for k, v in info.items():
                self.df.at[index, k] = v

        # Process details
        details = self.detailss(soup)
        if details:
            for k, v in details.items():
                self.df.at[index, k] = v

        # Process description
        description = self.descr(soup)
        self.df.at[index, "description"] = description

        # Process optional equipment
        optional = self.optionals(soup)
        if optional:
            for k, v in optional.items():
                self.df.at[index, k] = ",".join(v)

        # Process pricing information
        price = self.target_var(soup)
        self.df.at[index, "price"] = price

        currency = self.currrency(soup)
        self.df.at[index, "currency"] = currency

    @staticmethod
    def general_info(soup):
        info = {}
        try:
            items = soup.find("div", {"data-testid": "basic_information"}).find_all("div", class_="ooa-10m47vf eim4snj5")
            for item in items:
                cs = item.find_all(["p"])
                if len(cs) < 2:
                    continue
                info[cs[0].text.lower()] = cs[1].text.lower()
            return info
        except AttributeError:
            return None

    @staticmethod
    def detailss(soup):
        details = {}
        try:
            items = (
                soup.find("div", {"data-testid": "content-highlight-details-section"})
                .find("div", {"data-testid": "main-details-section"})
                .find_all("div", {"data-testid": "detail"})
            )
            for item in items:
                cs = item.find_all(["p"])
                details[cs[0].text.lower()] = cs[1].text.lower()
            return details
        except AttributeError:
            return None

    @staticmethod
    def descr(soup):
        try:
            return soup.find("div", {"data-testid": "content-description-section"}).find("div", class_="ooa-unlmzs e1jwj3574").text.lower()
        except AttributeError:
            return None

    @staticmethod
    def optionals(soup):
        equipment = {}
        try:
            items = (
                soup.find("div", {"data-testid": "content-equipments-section"})
                .find("div", class_="e1ldcu9s0 ooa-wja48h")
                .findChildren("div", recursive=False)
            )

            for k, v in zip(items[::2], items[1::2]):
                equipment[k.text.lower()] = [value.text.lower() for value in v.find_all("p", class_="evespt84 ooa-1i4y99d er34gjf0")]
            return equipment
        except AttributeError:
            return None

    @staticmethod
    def target_var(soup):
        try:
            return soup.find("h3", class_="offer-price__number evnmei44 ooa-1kdys7g er34gjf0").text.replace(" ", "")
        except (AttributeError, ValueError):
            return None

    @staticmethod
    def currrency(soup):
        try:
            return soup.find("p", class_="offer-price__currency evnmei45 ooa-m6bn4u er34gjf0").text.lower()
        except AttributeError:
            return None

    def save_to_csv(self, path):
        self.df.to_csv(path, index=False)

# Run

In [3]:
scraper = AutoVitScraper()
scraper.get_urls()
scraper.prepare_dataframe()
scraper.fetch_all_details()
scraper.save_to_csv("../data/dataset.csv")